In [2]:
#5章誤差逆伝播法
#5.4単純なレイヤの実装
#5.4.1乗算レイヤの実装
class MulLayer:
    def __init__(self):
        self.x=None
        self.y=None
        
    def forward(self,x,y):
        self.x=x
        self.y=y
        out=x*y
        return out
    
    def backward(self,dout):
        dx=dout*self.y #xとyをひっくり返す
        dy=dout*self.x
        return dx,dy

In [3]:
apple=100
apple_num=2
tax=1.1

#layer
mul_apple_layer=MulLayer()  
mul_tax_layer=MulLayer()

#forward
apple_price=mul_apple_layer.forward(apple,apple_num)
price=mul_tax_layer.forward(apple_price,tax)

print(price)#220

220.00000000000003


In [4]:
#backward
dprice=1
dapple_price,dtax=mul_tax_layer.backward(dprice)
dapple,dapple_num=mul_apple_layer.backward(dapple_price)
print(dapple,dapple_num,dtax)#2.2 110 200

2.2 110.00000000000001 200


In [5]:
#5.4.2加算レイヤの実装
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self,x,y):
        out=x+y
        return out
    
    def backward(self,dout):
        dx=dout*1
        dy=dout*1
        return dx,dy
    

In [6]:
apple=100
apple_num=2
orange=150
orange_num=3
tax=1.1

In [7]:
#layer
mul_apple_layer=MulLayer()
mul_orange_layer=MulLayer()
add_apple_orange_layer=AddLayer()
mul_tax_layer=MulLayer()

In [8]:
#forward
apple_price=mul_apple_layer.forward(apple,apple_num)
orange_price=mul_orange_layer.forward(orange,orange_num)
all_price=add_apple_orange_layer.forward(apple_price,orange_price)
price=mul_tax_layer.forward(all_price,tax)

In [9]:
#backward
dprice=1
dall_price,dtax=mul_tax_layer.backward(dprice)
dapple_price,dorange_price=add_apple_orange_layer.backward(dall_price)
dorange,dorange_num=mul_orange_layer.backward(dorange_price)
dapple,dapple_num=mul_apple_layer.backward(dapple_price)

In [10]:
print(price)
print(dapple_num,dapple,dorange,dorange_num,dtax)#110 2.2 3.3 165 650

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


In [11]:
#5.5活性化関数レイヤの実装
#5.5.1ReLUレイヤ
class Relu:
    def __init__(self):
        self.mask=None
        
    def forward(self,x):
        self.mask=(x<=0)
        out=x.copy()
        out[self.mask]=0
        return out
    
    def backward(self,dout):
        dout[self.mask]=0
        dx=dout
        return dx

In [17]:
import numpy as np

In [18]:
x=np.array([[1.0,-0.5],[-2.0,3.0]])
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [19]:
mask=(x<=0)
print(mask)

[[False  True]
 [ True False]]


In [20]:
#5.5.2 Sigmoidレイヤ
class Sigmoid:
    def __init__(self):
        self.out=None
        
    def forward(self,x):
        out=1/(1+np.exp(-x))
        self.out=out
        return out
    
    def backward(self,dout):
        dx=dout*(1.0-self.out)*self.out
        return dx

In [21]:
#5.6 Affine/Softmaxレイヤの実装
#5.6.1 Affineレイヤ
X=np.random.rand(2)#入力

In [22]:
W=np.random.rand(2,3)#重み

In [23]:
B=np.random.rand(3)#バイアス

In [24]:
X.shape #(2,)

(2,)

In [25]:
W.shape#(2,3)

(2, 3)

In [26]:
B.shape#(3,)

(3,)

In [27]:
Y=np.dot(X,W)+B

In [28]:
#5.6.2 バッチ版Affineレイヤ
X_dot_W=np.array([[0,0,0],[10,10,10]])

In [29]:
B=np.array([1,2,3])

In [30]:
X_dot_W

array([[ 0,  0,  0],
       [10, 10, 10]])

In [31]:
X_dot_W+B

array([[ 1,  2,  3],
       [11, 12, 13]])

In [32]:
dY=np.array([[1,2,3,],[4,5,6]])

In [33]:
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [34]:
dB=np.sum(dY,axis=0)

In [35]:
dB

array([5, 7, 9])

In [36]:
class Affine:
    def __init__(self,W,b):
        self.W=W
        self.b=b
        self.x=None
        self.dW=None
        self.db=None
        
    def forward(self,x):
        self.x=x
        out=np.dot(x,self.W)+self.b
        return out
    
    def backward(self,dout):
        dx=np.dot(dout,self.W.T)
        self.dW=np.dot(self.x.T,dout)
        self.db=np.sum(dout,axis=0)
        return dx

In [37]:
#5.6.3 Softmax-with-Lossレイヤ
class SoftmaxWithLoss:
    def __init__(self):
        self.loss=None #損失
        self.y=None #softmaxの出力
        self.t=None #教師データ(one-hot vector)
        
    def forward(self,x,t):
        self.t=t
        self.y=softmax(x)
        self.loss=cross_entropy_error(self.y,self.t)
        return self.loss
    
    def backward(self,dout=1):
        batch_size=self.t.shape[0]
        dx=(self.y-self.t)/batch_size
        return dx

In [43]:
#5.7誤差逆伝播法の実装
#5.7.2誤差逆伝播法に対応したニューラルネットワークの実装
import sys,os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

In [50]:
class TwoLayerNet:
    def __init__(self,input_size,hidden_size,output_size,weight_init_std=0.01):
        #重み
        self.params={}
        self.params['W1']=weight_init_std*np.random.randn(input_size,hidden_size)
        self.params['b1']=np.zeros(hidden_size)
        self.params['W2']=weight_init_std*np.random.randn(hidden_size,output_size)
        self.params['b2']=np.zeros(output_size)
        
        #レイヤの生成
        self.layers=OrderedDict()
        self.layers['Affine1']=Affine(self.params['W1'],self.params['b1'])
        self.layers['Affine2']=Affine(self.params['W2'],self.params['b2'])
        
        self.lastLayer=SoftmaxWithLoss()
        
    def predict(self,x):
        for layer in self.layers.values():
            x=layer.forward(x)
        return x
    
    #x:入力データ、t:教師データ
    def loss(self,x,t):
        y=self.predict(x)
        return self.lastLayer.forward(y,t)
    
    def accuracy(self,x,t):
        y=self.predict(x)
        y=np.argmax(y,axis=1)
        if t.ndim!=1:t=np.argmax(t,axis=1)
        accuracy=np.sum(y==t)/float(x.shape[0])
        return accuracy
    
    #x:入力データ、t:教師データ
    def numerical_gradient(self,x,t):
        loss_W=lambda W:self.loss(x,t)
        
        grads={}
        grads['W1']=numerical_gradient(loss_W,self.params['W1'])
        grads['b1']=numerical_gradient(loss_W,self.params['b1'])
        grads['W2']=numerical_gradient(loss_W,self.params['W2'])
        grads['b2']=numerical_gradient(loss_W,self.params['b2'])
        return grads
    
    def gradient(self,x,t):
        #forward
        self.loss(x,t)
        
        #backward
        dout=1
        dout=self.lastLayer.backward(dout)
        
        layers=list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout=layer.backward(dout)
            
        #設定
        grads={}
        grads['W1']=self.layers['Affine1'].dW
        grads['b1']=self.layers['Affine1'].db
        grads['W2']=self.layers['Affine2'].dW
        grads['b2']=self.layers['Affine1'].db
        
        return grads

In [52]:
#5.7.3誤差逆伝播法の勾配確認
import sys,os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

In [53]:
#でーたの読み込み
(x_train,t_train),(x_test,t_test)=load_mnist(normalize=True)
network=TwoLayerNet(input_size=784,hidden_size=50,output_size=10)
x_batch=x_train[:3]
t_batch=t_train[:3]
grad_numerical=network.numerical_gradient(x_batch,t_batch)
grad_backprop=network.gradient(x_batch,t_batch)

Done


Done


Done


Done
Converting train-images-idx3-ubyte.gz to NumPy Array ...


Done
Converting train-labels-idx1-ubyte.gz to NumPy Array ...
Done
Converting t10k-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting t10k-labels-idx1-ubyte.gz to NumPy Array ...
Done
Creating pickle file ...


Done!


ValueError: operands could not be broadcast together with shapes (3,10) (3,) 

In [54]:
#各重みの絶対誤差の平均を求める
for key in grad_numerical(np.abs(grad_backprop[key]-grad_numerical[key]))
    print(key+":"+str(diff))

SyntaxError: invalid syntax (<ipython-input-54-78ed222f605f>, line 2)

In [55]:
#誤差逆伝播法を使った学習
import sys,os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

In [56]:
#データの読み込み
(x_train,t_train),(x_test,t_test)=load_mnist(normalize=True,one_hot_label=True)

network=TwoLayerNet(input_size=784,hidden_size=50,output_size=10)

iters_num=10000
train_size=x_train.shape[0]
batch_size=100
learning_rate=0.1
train_loss_list=[]
train_acc_list=[]
test_acc_list=[]
iter_per_epoch=max(train_size/batch_size,1)

In [58]:
for i in range(iters_num):
    batch_mask=np.random.choice(train_size,batch_size)
    x_batch=x_train[batch_mask]
    t_batch=t_train[batch_mask]
    
    #誤差逆伝播法によって勾配を求める
    grad=network.gradient(x_batch,t_batch)
    
    #更新
    for key in ('W1','b1','W2','b2'):
        network.params[key]-=learning_rate*grad[key]
        
    loss=network.loss(x_batch,t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch==0:
        train_acc=network.accuracy(x_train,t_train)
        test_acc=network.accuracy(x_test,t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc,test_acc)

0.09035 0.0892


0.7819333333333334 0.7861


0.8741666666666666 0.8763


0.8980166666666667 0.9005


0.9082333333333333 0.9103


...

0.9387666666666666 0.9399


0.9414833333333333 0.9421


0.94365 0.9433


0.9452 0.9438


0.94655 0.9453
